Package Installations

In [2]:
!pip install kaggle

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 34.0 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 596 kB 73.8 MB/s 
     |████████████████████████████████| 6.6 MB 59.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 25.4 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [4]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [7]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 29% 12.0M/41.0M [00:00<00:00, 124MB/s]
100% 41.0M/41.0M [00:00<00:00, 224MB/s]


In [10]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [11]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [12]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [13]:
corpus["x"] = corpus["title"]

In [14]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [15]:
corpus = corpus[["x", "y"]]

In [16]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Twitter Dataset**

In [17]:
twitter_real_df = pd.read_csv('/content/real_news_tweets.csv')
twitter_real_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:35:25+00:00,1516334641610989571,Half-a-million Ukrainians have fled to the Lvi...,AP
1,1,2022-04-19 07:42:46+00:00,1516321393281015809,Afghan police say explosions targeting educati...,AP
2,2,2022-04-19 07:13:55+00:00,1516314133666349058,Russian forces have attacked along a broad fro...,AP
3,3,2022-04-19 06:40:38+00:00,1516305757674819584,There's stark public divide in Iran over Russi...,AP
4,4,2022-04-19 06:26:26+00:00,1516302180772364288,There's stark public divide in Iran over Russi...,AP
...,...,...,...,...,...
4995,4995,2022-02-24 00:13:22+00:00,1496639353820430337,The soldiers on the front line of the Ukraine-...,BBCWorld
4996,4996,2022-02-23 23:45:08+00:00,1496632246459158531,What's Putin's next move?,BBCWorld
4997,4997,2022-02-23 23:29:06+00:00,1496628212880875520,"Virgin Hyperloop, a futuristic train service e...",BBCWorld
4998,4998,2022-02-23 23:14:06+00:00,1496624437243879429,Metaverse app allows kids into virtual strip c...,BBCWorld


In [18]:
twitter_fake_df = pd.read_csv('/content/fake_news_tweets.csv')
twitter_fake_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:30:02+00:00,1516333287744102405,Grandpa Surprisingly Willing To Talk About Man...,TheOnion
1,1,2022-04-19 06:30:01+00:00,1516303084628099076,Nation Attempts To Fall Asleep By Doing Little...,TheOnion
2,2,2022-04-19 05:00:01+00:00,1516280436334473221,"When it comes to pip blips, Megan Fox takes th...",TheOnion
3,3,2022-04-19 04:30:01+00:00,1516272886314672130,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,TheOnion
4,4,2022-04-19 03:15:00+00:00,1516254005869096968,The Onion: Now on another part of your phone. ...,TheOnion
...,...,...,...,...,...
4995,4995,2021-10-02 14:00:00+00:00,1444301140703780868,Boss Warns All New Employees He Will Fire Them...,TheBabylonBee
4996,4996,2021-10-02 13:00:02+00:00,1444286051204403205,10 Tips For Making Sure Your Kids Are Terrifie...,TheBabylonBee
4997,4997,2021-10-02 12:21:32+00:00,1444276362865479696,Biden Says We Can Afford $3.5T Bill Because Ch...,TheBabylonBee
4998,4998,2021-10-02 02:00:00+00:00,1444119947274334211,Fly That Landed On Mike Pence During Debate Re...,TheBabylonBee


In [19]:
twitter_real_df['x'] = twitter_real_df['Text']
twitter_fake_df['x'] = twitter_fake_df['Text']

In [20]:
twitter_real_df['y'] = ['True'] * 5000
twitter_real_df['y'] = y_encoder.transform(twitter_real_df['y'])

In [21]:
twitter_fake_df['y'] = ['False'] * 5000
twitter_fake_df['y'] = y_encoder.transform(twitter_fake_df['y'])

In [22]:
twitter_real_df = twitter_real_df[["x", "y"]]
twitter_real_df.drop_duplicates(inplace=True)
twitter_real_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,The soldiers on the front line of the Ukraine-...,1
4996,What's Putin's next move?,1
4997,"Virgin Hyperloop, a futuristic train service e...",1
4998,Metaverse app allows kids into virtual strip c...,1


In [23]:
twitter_fake_df = twitter_fake_df[["x", "y"]]
twitter_fake_df.drop_duplicates(inplace=True)
twitter_fake_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [24]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [25]:
frame1 = [twitter_real_df, twitter_fake_df]
corpus_tweet = pd.concat(frame1)
corpus_tweet

,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


**Electra**

In [26]:
pip install simpletransformers

     |████████████████████████████████| 249 kB 25.7 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 10.1 MB 50.5 MB/s 
     |████████████████████████████████| 325 kB 70.4 MB/s 
     |████████████████████████████████| 1.8 MB 60.8 MB/s 
     |████████████████████████████████| 1.2 MB 61.8 MB/s 
     |████████████████████████████████| 181 kB 68.4 MB/s 
     |████████████████████████████████| 144 kB 72.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 136 kB 48.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.9 MB/s 
     |████████████████████████████████| 212 kB 56.7 MB/s 
     |████████████████████████████████| 127 kB 65.8 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 271 kB 63.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 111 kB 75.1 MB/s 
     |█████████████

In [27]:
from simpletransformers.classification import MultiLabelClassificationModel


# Create a ClassificationModel
model = MultiLabelClassificationModel('electra', 'google/electra-small-discriminator', num_labels=2)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForMultiLabelSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMultiLabelSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [28]:
# Create a TransformerModel with modified attributes
from simpletransformers.model import TransformerModel
model = TransformerModel('electra', 'google/electra-small-discriminator', num_labels=2, args={'learning_rate':1e-5, 'num_train_epochs': 3, 'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [29]:
train = corpus
test = corpus_tweet

In [30]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/38729 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/4842 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/4842 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/4842 [00:00<?, ?it/s]

(14526, 0.029703523130205467)

In [31]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/8917 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1115 [00:00<?, ?it/s]

In [32]:
print(result)
# print(model_outputs)
# print(wrong_predictions)

{'mcc': 0.7729476218344312, 'tp': 4382, 'tn': 3526, 'fp': 627, 'fn': 382, 'auroc': 0.9522240505533212, 'auprc': 0.9506892419552417, 'f1': 0.8868453515756419, 'acc': 0.886845351575642, 'eval_loss': 0.8475418597139064}
